In [1]:
import sempy.fabric as fabric
import base64
import json
import time


StatementMeta(, 188e7d07-4c34-4f9f-a3b0-4fe4190de569, 3, Finished, Available, Finished)

## Functions to get the Report definition and the report.json payload

In [2]:
def get_report_definition(workspaceId:str, itemId:str ):
    #Instantiate the client
    client = fabric.FabricRestClient()
    #request the report definition
    resp = client.post(f"v1/workspaces/{workspaceId}/items/{itemId}/getDefinition")

    ## TODO - need to be able to handle throttling for production use

    if resp.status_code == 200 :
        payload = json.loads(resp.content.decode('utf-8'))
    if resp.status_code == 202 :
        operation_resp = client.get(resp.headers["Location"])
        pcnt = json.loads(operation_resp.content.decode('utf-8'))["percentComplete"]
        percentComplete = 0 if pcnt is None else int(pcnt)

        while percentComplete < 100:
            time.sleep(1)
            operation_resp = client.get(resp.headers["Location"])
            pcnt = json.loads(operation_resp.content.decode('utf-8'))["percentComplete"]
            percentComplete = 0 if pcnt is None else int(pcnt)
        final_resp = client.get(operation_resp.headers["Location"])
        payload = json.loads(final_resp.content.decode('utf-8'))

    return payload

StatementMeta(, 188e7d07-4c34-4f9f-a3b0-4fe4190de569, 4, Finished, Available, Finished)

In [3]:
def get_report_json(payload):
    report_idx = -1
    idx = 0
    for part in payload["definition"]["parts"]:
        if part["path"] == "report.json":
            report_idx = idx
            break
        idx +=1

    decoded_string = base64.b64decode(payload["definition"]["parts"][report_idx]["payload"]).decode('utf-8')
    return (report_idx, decoded_string)

StatementMeta(, 188e7d07-4c34-4f9f-a3b0-4fe4190de569, 5, Finished, Available, Finished)

## Get all the reports in the current workspace

In [4]:
wsId = fabric.get_notebook_workspace_id()
reports = fabric.list_items('Report', wsId)

StatementMeta(, 188e7d07-4c34-4f9f-a3b0-4fe4190de569, 6, Finished, Available, Finished)

## Printing the value of the exportDataMode setting for all reports in a Workspace

In [5]:
for index, report in reports.iterrows():
    payload = get_report_definition(wsId,report['Id'] )
    reportIdx, reportJson = get_report_json(payload)
    pbir = json.loads(reportJson)
    exportDataMode = json.loads(pbir['config'])['settings'].get('exportDataMode',None)
    print(f"Report: {report['Display Name']} ({report['Id']}) ExportMode: {exportDataMode}") 


StatementMeta(, 188e7d07-4c34-4f9f-a3b0-4fe4190de569, 7, Finished, Available, Finished)

Report: AzSQL AdvWrksLt (549ecda6-0494-42ed-b5c0-cb3e95adb6e3) ExportMode: 1
Report: FabricMonitor-DQ (9eeceff7-571e-4ddd-81b4-a55b74356c8f) ExportMode: 1
Report: FabricMonitor-Import (fa5832d0-042a-444d-a889-055dfd0dbc2f) ExportMode: 1
Report: Adv Wrks Test (addd1b63-46e3-4a69-a641-8cd88071b496) ExportMode: 2
Report: FabricMonitorCustom Workspaces (880d095f-ac08-4bc1-8e2c-4ae573168105) ExportMode: 1
Report: FabricMonitorCustom2 Workspaces (c1b87198-ae20-4127-bd0f-228cb7779668) ExportMode: 1
Report: Adv Works DQ to SSAS (077cecd3-c8ca-4a71-a002-547ff484988a) ExportMode: 1
Report: AdvWrksLt Hybrid (bedb7a94-9eef-4ff4-9808-9b34aaf5ef8b) ExportMode: None


## Updating all reports to set Export: None

In [ ]:
client = fabric.FabricRestClient()
for index, report in reports.iterrows():
    # get the current definition
    payload = get_report_definition(wsId,report['Id'] )
    # find the report.json content
    report_idx, reportJson = get_report_json(payload)
    # update settings of 1 or 2 with 0
    # I'm using string replacement here as the settings are stored as a string inside the settings property
    # and I don't want to change the encoding
    new_str = reportJson.replace("\\\"exportDataMode\\\":1","\\\"exportDataMode\\\":0")
    new_str = new_str.replace("\\\"exportDataMode\\\":2","\\\"exportDataMode\\\":0")
    # Encode the string to bytes, then encode to Base64
    encoded_bytes = base64.b64encode(new_str.encode('utf-8'))
    encoded_str = encoded_bytes.decode('utf-8')
    payload["definition"]["parts"][report_idx]["payload"] = encoded_str

    # Post the update request
    ## TODO - need to be able to handle throttling for production use
    upd_resp = client.post(f"v1/workspaces/{wsId}/items/{report['Id']}/updateDefinition", json= payload)

    print(f"Report: {report['Display Name']} ({report['Id']}) Update Response: {upd_resp.status_code}") 